In [1]:
import csv
import xml.etree.ElementTree as ET

In [2]:
file = "C:/Users/bllanos/Desktop/project1/prueba1/contratos/MAT_sanitario/Contratos_rellenar_material_sanitario.csv"

print(file)

C:/Users/bllanos/Desktop/project1/prueba1/contratos/MAT_sanitario/Contratos_rellenar_material_sanitario.csv


In [3]:
root = ET.Element("contratos")

In [4]:
def ajustanumeros (numero):
    if len(numero) == 1:
        try:
            int(numero)
            return "0" + numero
        except:
            return numero
    else:
        return str(numero)

In [5]:
with open(file) as f:
    csv_reader = csv.DictReader(f,delimiter=";")
    for row in csv_reader:
        #print(row)
        contrato = ET.SubElement(root,"contrato")
        programaoperativo = 0
        fechaslicitacion = 0
        fechasformalizacion = 0
        subcontratistascount = 0 
        utescount = 0
        for k,v in row.items():
            if v == "":
                pass
            else:
                v = ajustanumeros(v)
                k = k.split(" ")[0]
                if k == "cod_medio_pub" or k == "f_publicacion" or k=="descripcion":
                    if fechaslicitacion == 0:
                        licitaciones = ET.SubElement(contrato,"fechas_publicacion_licitacion")
                        fechaslicitacion = 1
                    if k == "cod_medio_pub":
                        licitacion = ET.SubElement(licitaciones,"fecha_publicacion_licitacion")
                    elemento = ET.SubElement(licitacion,f"{k}")
                    elemento.text= f"{v.replace(',','.')}"
                    
                elif k == "codigo_poperativo":
                    if programaoperativo == 0:
                        codigos = ET.SubElement(contrato,"programas_operativos")
                        codigo = ET.SubElement(codigos,"programa_operativo")
                        programaoperativo = 1
                    elemento = ET.SubElement(codigo,f"{k}")
                    elemento.text= f"{v.replace(',','.')}"
                    
                elif k == "cod_medio_pub-formalizacion" or k == "f_publicacion-formalizacion" or k == "descripcion-formalizacion":
                    if fechasformalizacion == 0:
                        formalizaciones = ET.SubElement(contrato,"fechas_publicacion_formalizacion")
                        formalizacion = ET.SubElement(formalizaciones,"fecha_publicacion_formalizacion")
                        fechasformalizacion = 1
                    k = k.split("-")[0]
                    elemento = ET.SubElement(formalizacion,f"{k}")
                    elemento.text= f"{v.replace(',','.')}"
                    
                elif k == "cod_subcontratista" or k == "titulo_contrato" or k == "importe":
                    if subcontratistascount == 0:
                        subcontratistas = ET.SubElement(contrato,"subcontratistas")
                        subcontratista = ET.SubElement(subcontratistas,"subcontratista")
                        subcontratistascount = 1
                    elemento = ET.SubElement(formalizacion,f"{k}")
                    elemento.text= f"{v.replace(',','.')}"
                    
                elif k == "cod_org_adjudicatario":
                    if utescount == 0:
                        utes = ET.SubElement(contrato,"utes")
                        ute = ET.SubElement(utes,"ute")
                        subcontratistascount = 1
                    elemento = ET.SubElement(formalizacion,f"{k}")
                    elemento.text= f"{v.replace(',','.')}"
                    
                else:
                    elemento = ET.SubElement(contrato,f"{k}")
                    elemento.text= f"{v.replace(',','.')}"
    tree = ET.ElementTree(root)
    tree.write("C:/Users/bllanos/Desktop/project1/prueba1/contratos/MAT_sanitario/Contratos_rellenar_material_sanitario.xml", encoding = "UTF-8", xml_declaration=True,)
        
        
        

In [6]:
with open(file) as f:
    csv_reader = csv.DictReader(f,delimiter=";")
    for row in csv_reader:
        for k,v in row.items():
            print(k, ajustanumeros(v))

cod_contrato_benef (7) MATS_nr7
nombre (8) Suministro de solución higienizante
titulo_largo (9) Suministro de solución higienizante
codigo_poperativo (10) PE
cod_normativa (11) 99
normativa_especifica (12) Contratación de emergencia en virtud del art. 16 del Real Decreto-Ley 7/2020 de 12 de marzo, por el que se adoptan medidas urgentes para responder al impacto económico del COVID-19 y el artículo 120 de la Ley 9/2017 de Contratos del Sector Público.
cod_tipocon_norma (13) 03
objeto_contrato (14) Suministro de solución higienizante
cod_proc_adj (15) 01
caracteristicas_especiales (16) A
tramitacion (17) E
cod_medio_pub (18) BOE
f_publicacion (18) 27/04/2020
descripcion (18) https://www.boe.es/diario_boe/txt.php?id=BOE-B-2020-13005
f_formalizacion (19) 22/04/2020
cod_medio_pub (20) 
f_publicacion (20) 
descripcion (20) 
imp_licitacion (21) 1908323,67
iva_general  S
iva_aplicable 
duracion_contrato (22) 
valor_estimado (23) 1577127
importe_adjudicado (24) 1577127
presupuesto_vigente (25) 